### Related Course

In [ ]:
import pandas as pd
import numpy as np
import joblib
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity

# === Load saved data and model ===
courses = pd.read_csv('udemy_courses (1).csv')  # Make sure 'index' column exists if used
tfidf = joblib.load('tfidf_vectorizer.pkl')
course_embeddings = np.load('course_embeddings.npy')  # Shape: (num_courses, embedding_dim)
encoder_model = tf.keras.models.load_model('course_encoder_model.keras')  # .keras format

# === Define recommendation function ===
def recommend_related(course_id, courses_df, embeddings, top_n=5):
    if course_id not in courses_df['id'].values:
        raise ValueError(f"Course ID {course_id} not found in dataset.")

    idx = courses_df[courses_df['id'] == course_id].index[0]
    course_vec = embeddings[idx].reshape(1, -1)

    similarities = cosine_similarity(course_vec, embeddings).flatten()
    courses_df['similarity'] = similarities

    recommendations = courses_df[courses_df['id'] != course_id] \
        .sort_values(by='similarity', ascending=False) \
        .head(top_n)

    return recommendations[['id', 'title', 'similarity']]

# === Example usage ===
recommended = recommend_related(course_id=567828, courses_df=courses, embeddings=course_embeddings, top_n=5)
print(recommended)


In [8]:
import requests
import json

# The URL of your Flask server (make sure Flask is running locally or on the server)
url = "http://127.0.0.1:5000/recommend-by-id"

# Sample data to send (example course_id)
data = {
    "course_id": 221674  # Replace this with an actual course ID you want to test
}

# Send the POST request to the Flask route
response = requests.post(url, json=data)

# Print the response from the Flask server
if response.status_code == 200:
    print("Recommended Courses:", json.dumps(response.json(), indent=4))
else:
    print(f"Error: {response.status_code} - {response.json()}")


JSONDecodeError: Expecting value: line 1 column 1 (char 0)